# 여러가지 방법의 모델 구축

## keras
1. sequential API

2. functional API 

3. class definition



## 배울 것

1. weight 저장방법
2. 모델 전체 저장방법 -> serializing model
    - weight
    - 모델 구조
    - model.compile() 로 정의한 학습 방법
    - optimizer, state

# 4일차 정리

## keras

텐서플로우를 더 쉽고 간편하게 사용하기 위한 API, API란 필요한 기능이나 도구들을 미리 구현하여 제공하는 것




## Layer 구현

`tf.keras.layers.Layer` 클래스를 상속받아서 하위 클래스를 구현하는 방식

1. `__init__` 함수를 통해 레이어 인스턴스 생성 시점에 수행할 동작 정의, output_shapes 정의


2. `build` 함수를 통해 인스턴스 최초 호출시, 입력텐서 X에 맞게 가중치 텐서를 정의


3. `call` 함수를 통해 인스턴스 호출시, 수행할 연산을 정의


## Model 구현


Layer와 동일한 방식으로 구현


### Sequential API


### Functional API


### Subclass  


In [3]:
import tensorflow as tf

In [8]:
# 선형회귀 모델을 keras로 구현하기
from tensorflow.keras import Model

class LinearModelKeras(Model):
    
    def __init__(self):
        
        super().__init__()

        # 가중치 W, 바이어스b 변수 텐서를 정의 
        # 이미 우리가 x,y의 shape를 이미 알고 있기 때문에 

        # 초기값을 정해야합니다. -> 단일 값 텐서를 정의하였다. 
        # 입력 텐서도 단일 값이기 때문에 

        # 실제값 3으로 가까워질 텐서
        self.w = tf.Variable(44, dtype = tf.float32, name = 'w')

        # 실제값 2와 가까워질 텐서
        self.b = tf.Variable(1, dtype = tf.float32, name = 'b')

        
    def call(self, X):
        # 소문자 x가 아닌 대문자 X로 변경
        y = self.w *  X + self.b

        return y

In [9]:
# 모델 인스턴스 생성
keras_model = LinearModelKeras()

In [10]:
TRUE_W = 3.0
TRUE_b = 2.0
 
# 학습 데이터의 크기는 1000
TRAIN_SIZE = 1000

# 정규 분포 상에서 (0, 1)사이에서 랜덤하게 값을 뽑습니다.  
x = tf.random.normal(shape = [TRAIN_SIZE])

# 기존 데이터와는 조금 차이를 주기 위한 노이즈
noise = tf.random.normal(shape = [TRAIN_SIZE])


y = x * TRUE_W + TRUE_b + noise

In [11]:
keras_model.compile(
    optimizer = tf.keras.optimizers.SGD(),
    loss = tf.keras.losses.mean_squared_error
)

In [13]:
# 학습을 시작
keras_model.fit(x, y, epochs = 100, batch_size = 30)

Epoch 1/100
34/34 [==============================] - 0s 499us/step - loss: 17.4897
Epoch 2/100
34/34 [==============================] - 0s 469us/step - loss: 5.1801
Epoch 3/100
34/34 [==============================] - 0s 469us/step - loss: 2.0947
Epoch 4/100
34/34 [==============================] - 0s 469us/step - loss: 1.2994
Epoch 5/100
34/34 [==============================] - 0s 440us/step - loss: 1.0967
Epoch 6/100
34/34 [==============================] - 0s 469us/step - loss: 1.0464
Epoch 7/100
34/34 [==============================] - 0s 469us/step - loss: 1.0321
Epoch 8/100
34/34 [==============================] - 0s 464us/step - loss: 1.0279
Epoch 9/100
34/34 [==============================] - 0s 469us/step - loss: 1.0272
Epoch 10/100
34/34 [==============================] - 0s 469us/step - loss: 1.0272
Epoch 11/100
34/34 [==============================] - 0s 469us/step - loss: 1.0273
Epoch 12/100
34/34 [==============================] - 0s 499us/step - loss: 1.0271
Epoch 13/100

In [14]:
# 모델이 올바르게 학습되었는가 
keras_model.trainable_variables

[<tf.Variable 'w:0' shape=() dtype=float32, numpy=3.0249505>,
 <tf.Variable 'b:0' shape=() dtype=float32, numpy=2.048449>]

In [15]:
from tensorflow.keras.datasets import mnist

In [16]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [17]:
# 행렬 형태의 이미지를 배열 형태의 이미지로 변환하는 작업
x_train = x_train.reshape(-1, 28 * 28 ).astype('float32') / 255
x_test = x_test.reshape(-1, 28 * 28 ).astype('float32') / 255

In [21]:
x_train.shape

(60000, 784)

# Keras로 모델 구현 리뷰

In [18]:
# Subclass 방법으로 모델 구현
# Model이라는 클래스를 상속받아 구현

from tensorflow.keras import Model

class SubModel(Model):
    
    
    def __init__(self):
        super().__init__()
        
        # fully connected layer 1개 추가
        # 64라는 의미는 해당 layer의 output shape가 (, 64)라는 의미이다.
        self.dense1 = tf.keras.layers.Dense(64)
        
        # fully connected layer 1개 추가
        # 10라는 의미는 해당 layer의 output shape가 (, 10)라는 의미이다.
        # 최종적으로 분류해야할 카테고리가 0-9까지의 숫자이기 때문이다.
        self.dense2 = tf.keras.layers.Dense(10)
        
        
    def call(self, X):
        
        # 첫번째 fully connected layer를 통과
        # 해당 레이어의 output shape (None, 64)
        x = self.dense1(X)
        
        # 두 fully connected layer 사이에 활성화함수를 추가하기 위해
        x = tf.nn.relu(x)
        
        # 두번째 fully connected layer를 통과
        # 해당 레이어의 output shape (None, 10)
        y = self.dense2(x)
        
        return y

In [19]:
sub_model = SubModel()

## Functional API

### Input 크기 정의

Input의 shape는 사용할 데이터의 shape와 동일하다. 사용 데이터의 shape중에 첫 번째 차원의 크기 60000은 단순히 데이터의 갯수를 의미한다.Input의 경우, 1개의 데이터의 shape를 전달해주면 되므로 (60000, 784) -> (784) 만 사용하면 된다. (3000, 128, 128, 3)의 데이터가 있다면, (128, 128, 3)만 Input에 넣어주면 된다.

In [22]:
# Functional

# Input을 정의
inputs = tf.keras.Input( 28 * 28 )

# Dense클래스의 layer인스턴스를 생성하고 즉시 inputs라는 텐서를 통해 호출하는 방식
# fly라는 변수는 fully connected layer 인스턴스 생성
# fly =  tf.keras.layers.Dense(64)
# x = fly(inputs)

# 첫번째 fully connected layer 통과
x =  tf.keras.layers.Dense(64)(inputs)

# 활성화 함수 통과
x = tf.nn.relu(x)

# 두번째 fully connected layer 통과
y = tf.keras.layers.Dense(10)(x)

func_model = tf.keras.Model(inputs = inputs, outputs = y)

In [23]:
func_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                50240     
_________________________________________________________________
tf_op_layer_Relu (TensorFlow [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Sequential API
seq_model = tf.keras.Sequential([
    # 통과할 layer를 배열로 제공
    tf.keras.layers.Dense(64),
    # 활성화 함수 레이어
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(10)
])

### 3가지 모델에 대해서 학습

In [33]:
# functional API로 구현한 모델
func_model
# subclass로 구현한 모델
sub_model
# sequential API로 구현한 모델
seq_model

In [45]:
# 이전
func_model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [ ]:
sub_model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [48]:
seq_model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [46]:
# 학습 
# epochs 30이라는 의미는 현재 학습 데이터를 총 30번 사용하겠다.
# batch_size가 100이라는 의미는 가중치 갱신시, 100개씩 사용하여 가중치를 반영시키겠다.
# step  60000 / 100 => 600
func_model.fit(x_train, y_train, epochs = 30, batch_size = 100)

Epoch 1/30
600/600 [==============================] - 0s 503us/step - loss: 0.3895 - accuracy: 0.8940
Epoch 2/30
600/600 [==============================] - 0s 502us/step - loss: 0.1890 - accuracy: 0.9466
Epoch 3/30
600/600 [==============================] - 0s 511us/step - loss: 0.1429 - accuracy: 0.9585
Epoch 4/30
600/600 [==============================] - 0s 515us/step - loss: 0.1145 - accuracy: 0.9671
Epoch 5/30
600/600 [==============================] - 0s 510us/step - loss: 0.0962 - accuracy: 0.9720
Epoch 6/30
600/600 [==============================] - 0s 510us/step - loss: 0.0823 - accuracy: 0.9755
Epoch 7/30
600/600 [==============================] - 0s 509us/step - loss: 0.0708 - accuracy: 0.9790
Epoch 8/30
600/600 [==============================] - 0s 509us/step - loss: 0.0627 - accuracy: 0.9815
Epoch 9/30
600/600 [==============================] - 0s 507us/step - loss: 0.0543 - accuracy: 0.9843
Epoch 10/30
600/600 [==============================] - 0s 510us/step - loss: 0.049

In [49]:
seq_model.fit(x_train, y_train, epochs = 30, batch_size = 100)

Epoch 1/30
600/600 [==============================] - 0s 503us/step - loss: 0.3861 - accuracy: 0.8940
Epoch 2/30
600/600 [==============================] - 0s 510us/step - loss: 0.1883 - accuracy: 0.9465
Epoch 3/30
600/600 [==============================] - 0s 519us/step - loss: 0.1449 - accuracy: 0.9576
Epoch 4/30
600/600 [==============================] - 0s 513us/step - loss: 0.1186 - accuracy: 0.9653
Epoch 5/30
600/600 [==============================] - 0s 510us/step - loss: 0.0992 - accuracy: 0.9711
Epoch 6/30
600/600 [==============================] - 0s 516us/step - loss: 0.0856 - accuracy: 0.9747
Epoch 7/30
600/600 [==============================] - 0s 512us/step - loss: 0.0738 - accuracy: 0.9788
Epoch 8/30
600/600 [==============================] - 0s 513us/step - loss: 0.0647 - accuracy: 0.9811
Epoch 9/30
600/600 [==============================] - 0s 512us/step - loss: 0.0573 - accuracy: 0.9833
Epoch 10/30
600/600 [==============================] - 0s 510us/step - loss: 0.051

In [50]:
sub_model.fit(x_train, y_train, epochs = 30, batch_size = 100)

Epoch 1/30
600/600 [==============================] - 0s 505us/step - loss: 0.3958 - accuracy: 0.8909
Epoch 2/30
600/600 [==============================] - 0s 505us/step - loss: 0.2002 - accuracy: 0.9432
Epoch 3/30
600/600 [==============================] - 0s 506us/step - loss: 0.1524 - accuracy: 0.9564
Epoch 4/30
600/600 [==============================] - 0s 517us/step - loss: 0.1236 - accuracy: 0.9646
Epoch 5/30
600/600 [==============================] - 0s 505us/step - loss: 0.1038 - accuracy: 0.9701
Epoch 6/30
600/600 [==============================] - 0s 508us/step - loss: 0.0890 - accuracy: 0.9743
Epoch 7/30
600/600 [==============================] - 0s 517us/step - loss: 0.0775 - accuracy: 0.9782
Epoch 8/30
600/600 [==============================] - 0s 508us/step - loss: 0.0685 - accuracy: 0.9798
Epoch 9/30
600/600 [==============================] - 0s 505us/step - loss: 0.0599 - accuracy: 0.9828
Epoch 10/30
600/600 [==============================] - 0s 500us/step - loss: 0.053

In [51]:
# 학습이 잘되었는지 평가
func_model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 362us/step - loss: 0.1088 - accuracy: 0.9740


[0.10876535624265671, 0.9739999771118164]

In [52]:
seq_model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 360us/step - loss: 0.0978 - accuracy: 0.9755


[0.09781114757061005, 0.9754999876022339]

In [54]:
sub_model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 356us/step - loss: 0.1148 - accuracy: 0.9736


[0.11475356668233871, 0.9735999703407288]

## learning rate가 과도하게 높은 경우, 제대로 학습이 진행되지 않는다.

learning rate가 실제 필요한 값보다 높게 설정하는 경우, 가중치 값을 수렴하는데 있어서 제대로 수렴하지 못하는 경우가 발생한다.

Adam optimizer의 learning_rate의 기본 값은 0.001인데, 0.1로 100배의 높은 값을 사용하였기 때문에 제대로 학습되지 않았다.

In [102]:
# Sequential API
seq_model = tf.keras.Sequential([
    # 통과할 layer를 배열로 제공
    tf.keras.layers.Dense(64),
    # 활성화 함수 레이어
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(10)
])

In [103]:
seq_model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [104]:
seq_model.fit(x_train, y_train, epochs = 30, batch_size = 100)

Epoch 1/30
600/600 [==============================] - 0s 505us/step - loss: 0.3926 - accuracy: 0.8926
Epoch 2/30
600/600 [==============================] - 0s 507us/step - loss: 0.1936 - accuracy: 0.9445
Epoch 3/30
600/600 [==============================] - 0s 515us/step - loss: 0.1472 - accuracy: 0.9578
Epoch 4/30
600/600 [==============================] - 0s 523us/step - loss: 0.1199 - accuracy: 0.9657
Epoch 5/30
600/600 [==============================] - 0s 511us/step - loss: 0.1014 - accuracy: 0.9698
Epoch 6/30
600/600 [==============================] - 0s 511us/step - loss: 0.0868 - accuracy: 0.9747
Epoch 7/30
600/600 [==============================] - 0s 511us/step - loss: 0.0761 - accuracy: 0.9778
Epoch 8/30
600/600 [==============================] - 0s 517us/step - loss: 0.0673 - accuracy: 0.9805
Epoch 9/30
600/600 [==============================] - 0s 516us/step - loss: 0.0598 - accuracy: 0.9827
Epoch 10/30
600/600 [==============================] - 0s 514us/step - loss: 0.053

# 모델의 가중치 값만 저장하거나 로딩하기

오직 가중치만 저장하고 가져오기만 가능하도록 하고싶을 수가 있다. 이럴 때가 있나 싶지만

1. 추론즉, 결과값 반환만을 수행하고 싶을 때 사용한다. 이 때는 학습이 필요하지 않기 때문에 compile option, optimizer들이 필요가 없다.
2. 전이학습을 진행하는 경우에 사용한다. 이 전 모델의 상태만 재사용하여 새로운 모델을 트레이닝하는 경우, 즉, 이전 모델의 compile option, optimizer가 필요하지 않지 않기 때문에 가중치만 가져와서 사용한다.


## 전이학습

> 기존의 데이터를 통해 구현한 문제 해결 방식들을 연관성이 존재하는 다른 문제에 적용할 수 있도록 하는 머신러닝의 한 방법

이 때, 가중치 값만 가져오기 떄문에 모델의 아키텍처는 구현이 되어있거나 선언되어있어야한다.

* 가중치가 존재하지 않는 relu, dropout, batchNormalization 과같은 layer가 있더라도 가중치의 값을 로딩할 수 있다.



### 메모리 상에서 가중치 전달

```python
# 해당 모델에서 weight를 가져온다.
model.get_weights()


#
new_model.set_weights(<weight>)

```


### 디스크에 가중치 저장, 로드

```python
model.save_weights(<save_format>)

```

모델을 저장하고 다시 로드할 때, 반드시 모델을 구현하는 방식 (subclass, functional API, sequential API)을 동일하게 사용해야한다.

# weight 저장방법


저장할 때는 동일한 방법으로 모델을 구현하고 해당 모델에 가중치를 로드해야한다.

## class subclass 방식의 모델의 가중치를 전달

1. 변수를 통한 가중치 전달. -> NO

2. 파일을 통한 가중치 전달. -> YES

In [61]:
# 1. 메모리(코드) 상에서 모델의 가중치를 전달 -> 변수 'sub_weights'에 가중치를 저장
sub_weights = sub_model.get_weights()

In [63]:
# 2. 실제 파일로 모델의 가중치를 전달
sub_model.save_weights('./weights/sub')

In [71]:
# 파일로 모델의 가중치를 로드
# 동일한 형태의 모델을 새로 생성
sub_new_model = SubModel()

In [66]:
sub_new_model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [67]:
# 초기값을 가지고 있기 때문에 정확도가 매우 낮거나 실행되지 않을 것.
sub_new_model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 368us/step - loss: 2.3852 - accuracy: 0.0808


[2.3852014541625977, 0.08079999685287476]

In [68]:
sub_new_model.load_weights('./weights/sub')

In [69]:
sub_new_model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 354us/step - loss: 0.1148 - accuracy: 0.9736


[0.11475356668233871, 0.9735999703407288]

In [70]:
# 변수를 통해 가중치 전달이 가능한지
sub_weights

[array([[ 0.02248818, -0.06485312,  0.03017422, ...,  0.01686268,
          0.00288638,  0.01042879],
        [ 0.03235076,  0.08181152,  0.06495798, ..., -0.05207455,
         -0.07318275,  0.03650863],
        [ 0.06364905,  0.01289119, -0.0355127 , ...,  0.05621331,
         -0.00037332,  0.0554458 ],
        ...,
        [-0.04104753,  0.03669202, -0.001722  , ..., -0.02938191,
          0.03350545,  0.01162734],
        [-0.05755827, -0.05904122,  0.03956515, ..., -0.08326103,
          0.06350818,  0.02566323],
        [-0.02726152, -0.01475883, -0.01435865, ...,  0.04154914,
         -0.02404592, -0.02891355]], dtype=float32),
 array([ 0.37633452, -0.05193192, -0.08608557,  0.04161313, -0.04378584,
        -0.03874684,  0.26989302, -0.30805224, -0.16692297, -0.21364735,
        -0.06521977,  0.14161904, -0.14672603,  0.28632572, -0.09810597,
         0.24433994, -0.03250898,  0.3001712 ,  0.1140523 , -0.06633083,
        -0.04890596,  0.18620513, -0.01224095,  0.13151869, -0.066

In [72]:
# 파일로 모델의 가중치를 로드
# 동일한 형태의 모델을 새로 생성
sub_new_model = SubModel()

In [73]:
# 변수를 통한 가중치 전달은 되지 않는다.
sub_new_model.load_weights(sub_weights)

AttributeError: 'list' object has no attribute 'endswith'

## Sequential API에 가중치 전달


1. 변수를 통한 가중치 전달. -> NO

2. 파일을 통한 가중치 전달. -> YES

In [74]:
# 1. 메모리(코드) 상에서 모델의 가중치를 전달 -> 변수 'seq_weights'에 가중치를 저장
seq_weights = seq_model.get_weights()

In [75]:
# 2. 실제 파일로 모델의 가중치를 전달
seq_model.save_weights('./weights/seq')

In [76]:
# Sequential API
seq_new_model = tf.keras.Sequential([
    # 통과할 layer를 배열로 제공
    tf.keras.layers.Dense(64),
    # 활성화 함수 레이어
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(10)
])

In [77]:
seq_new_model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [78]:
# 초기값을 가지고 있기 때문에 정확도가 매우 낮거나 실행되지 않을 것.
seq_new_model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 359us/step - loss: 2.3984 - accuracy: 0.0904


[2.39835262298584, 0.09040000289678574]

In [79]:
# 파일을 통해 가중치 로드
seq_new_model.load_weights('./weights/seq')

In [89]:
# 초기값을 가지고 있기 때문에 정확도가 매우 낮거나 실행되지 않을 것.
seq_new_model.evaluate(x_test, y_test)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [81]:
# Sequential API
seq_new_model = tf.keras.Sequential([
    # 통과할 layer를 배열로 제공
    tf.keras.layers.Dense(64),
    # 활성화 함수 레이어
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(10)
])

In [82]:
# 변수를 통한 가중치 전달은 되지 않는다.
seq_new_model.load_weights(sub_weights)

AttributeError: 'list' object has no attribute 'endswith'

# Functional API

1. 변수를 통한 가중치 전달. -> YES

2. 파일을 통한 가중치 전달. -> YES

#### 과제로 해보세요.

# model 전체 저장방법

```

model.save('')
```

#### 반드시 디렉토리 형태로 저장된다.



```
model = keras.models.load_model('')
```

In [121]:
seq_model.save('./models/seq')

In [123]:
# 가중치 전달을 통한 모델을 사용하는 경우, 반드시 모델이 선언되어 있어야 했다.

model = tf.keras.models.load_model('./models/seq')

In [124]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 64)                50240     
_________________________________________________________________
re_lu_7 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________


In [115]:
model.weights

[<tf.Variable 'dense_24/kernel:0' shape=(784, 64) dtype=float32, numpy=
 array([[ 0.01094966,  0.02156093,  0.07458273, ..., -0.05185828,
          0.01536532, -0.01333644],
        [-0.06121151, -0.05409407,  0.0052647 , ...,  0.07546673,
         -0.05898435, -0.05778118],
        [-0.07429571, -0.06967273, -0.01311747, ..., -0.04415247,
         -0.06222134,  0.00702934],
        ...,
        [ 0.07957853,  0.04390956,  0.05781852, ..., -0.02822533,
         -0.07199149,  0.03036475],
        [ 0.00567599, -0.03325198,  0.05133317, ..., -0.06005932,
         -0.01420709, -0.06091733],
        [-0.05108691,  0.02119628,  0.04003884, ...,  0.08309915,
         -0.02316669, -0.02904134]], dtype=float32)>,
 <tf.Variable 'dense_24/bias:0' shape=(64,) dtype=float32, numpy=
 array([ 0.05264931, -0.38654673,  0.07460687,  0.4039744 , -0.0259308 ,
        -0.24682625,  0.2589934 , -0.07147194,  0.00321197, -0.01497718,
        -0.18127425,  0.05815915,  0.02055575,  0.25439224, -0.02408221,


In [127]:
# 기존의 불러오는 모델과 동일한 학습루프를 정의해줘야 동일한 정확도를 확인할 수 있다.
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [129]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 367us/step - loss: 0.1065 - accuracy: 0.9751


[0.10649846494197845, 0.9750999808311462]

## 위의 모델 저장방법을 사용하는 경우, 모델 + 모델에 대한 설정까지 포함한다


- 모델의 아키텍처 (구조)
- 모델의 가중치 (내용물)
- 컴파일 정보 (구동방식)
- optimizer, 현재까지 학습한 상태 (현재의 위치)


## 저장 가능 형식은 Tensorflow SavedModel, Keras H5 형식

- 디폴트는 SavedModel 형식이다.



## SavedModel 형식이 사용자 정의 Object를 다루는 방법

- class name, call function, losses, weights, config를 저장한다.
- 사용자 정의 class가 코드에 존재하지 않아도 가능하다. (애초에 클래스 이름과 call function이 존재하기 때문에)
- 하지만, call function 만을 저장하기 때문에 학습 및 평가는 가능하지만 다른 method는 사용하지 못한다.


# Keras H5

H5 형식은 다음과 같이 저장하며

- 모델의 아키텍쳐
- 모델의 가중치
- 컴파일 정보


saved model과의 차이는 

- 사용자 정의 object를 저장하지 않는다는 것
- add_loss(), add_metric()으로 추가한 것은 저장되지 않는다.
- Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model.
> 즉, Functional model or a Sequential model 에 대해서만 사용이 가능하다.  
> 내부에서 subclass layer의 사용도 하지 못한다.


# 아키텍처 저장

SubClass 모델이 아닌 아래의 방식으로 구현하였을 때만 사용이 가능하다.
- 함수형
- Sequential API


```
get_config()


from_config()


tf.keras.models.model_to_json()


tf.keras.models.model_from_json()
```
